In [14]:
import pandas as pd

In [ ]:
# Обучение модели, нужен размеченый ждейсон

# !python -m spacy download ru_core_news_sm
# !pip install pymorphy2
# !pip install spacy
import json
import spacy
import random
from spacy.training.example import Example
import pymorphy2
nlp = spacy.load("ru_core_news_sm")
TRAINING_DATA = []
# data = json.loads(open("jjson.json", "r").read())

# for i in data:
#   TRAINING_DATA.append((i["test"], {"entities": i["entities"]}))
# print(TRAINING_DATA, sep='\n')

TRAINING_DATA = []
data = json.loads(open("json_1.0.json", "r").read())

for i in data:
  TRAINING_DATA.append((i["text"], {"entities": i["entities"]}))
print(TRAINING_DATA, sep='\n')
ner = nlp.get_pipe("ner")
ner.add_label("EQUIPMENT")
disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*disable_pipes):
    optimizer = nlp.begin_training()
    for itn in range(35):
        random.shuffle(TRAINING_DATA)
        losses = {}
        for text, annotations in TRAINING_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], losses=losses)
        print(losses)
input()
doc = nlp("1. ОБЩИЕ ТРЕБОВАНИЯ\n1.1. Общие требования - по ГОСТ 9733.0-83.\n2. АППАРАТУРА, МАТЕРИАЛЫ\n(раздел 2 в ред. Изменения N 1, введенного Постановлением\nГосстандарта СССР от 26.09.1986 N 2912)\n2.1. Прибор для испытания устойчивости окраски к трению, обеспечивающий перемещение смежной хлопчатобумажной ткани или элементарной пробы на расстояние 100 мм, с нагрузкой 9 Н и диаметром трущего стержня 16 мм.\n2.2. Смежная хлопчатобумажная ткань по ГОСТ 9733.0-83.\n2.3. Серые шкалы для оценки закрашивания и изменения окраски по ГОСТ 9733.0-83.\n3. ОТБОР И ПОДГОТОВКА ПРОБ\n(раздел 3 в ред. Изменения N 1, введенного Постановлением\nГосстандарта СССР от 26.09.1986 N 2912")
morph = pymorphy2.MorphAnalyzer()
print([(ent.text, ent.label_) for ent in doc.ents])
# [('Прибор для испытания устойчивости окраски к трению', 'EQUIPMENT'), ('Серые шкалы для оценки закрашивания и изменения окраски по ГОСТ 9733.0-83', 'EQUIPMENT')]

In [ ]:

# nlp_tmp = spacy.load("nlp_model1")
# for i in open("./gosts/ГОСТ 31870-2012.txt", "r").readlines():
#     doc = nlp_tmp(i)
#     if len(doc.ents) != 0:
#         print([(ent.text, ent.label_) for ent in doc.ents])

In [25]:
import random
df = pd.read_csv('dataset.csv')
print(df.shape)
# print(df.shape)
# col = df.columns
# print(col[5])
# print(*df[col[1]][:100], sep='\n')
s = set(df[col[-1]])
# s3 =s
# print(len(s))
# print(*(s), sep="\n")
a = []
for i in s:
    for j in i.split(";"):
        if len(j) > 2:
            j = j.strip()
            a.append({
                "text": j + ' ',
                "entities": [[0, len(j), "EQUIPMENT"]]
            })
tmp = []
for i in range(15):
    s = open(f"train_{i}.txt", "r").read()
    cnn = 0
    for i in a:
        if i["text"][:-1] in s:
            cnn += 1
            s = s.replace(i["text"][:-1], " ")

    f = open(f"train_-1.txt", "w")
    f.write(s)
    f.close()
    for j in open(f"train_-1.txt", "r").readlines():
        if len(j) > 2 and len(j.split()) > 2:
            j = j.strip()
            tmp.append(j)
            # a.append({
            #     "text": j + ' ',
            #     "entities": []
            # })
print(len(tmp))
random.shuffle(tmp)
print(len(a))
j = 0
for i in range(len(a)):
    a[i]["text"] = tmp[j] + ' ' + a[i]["text"] + ' ' + tmp[j+1] + ' '
    a[i]["entities"][0][0] += 1 + len(tmp[j])
    a[i]["entities"][0][1] += 1 + len(tmp[j])
    j += 2
print(a[:5])
f = open("secjson.json", "w")
jj = json.dumps(a)
print(jj)
f.write(jj)
f.close()
len(a)

(12825, 6)
5245
1994
[{'text': 'ISO 11885:2007. Water quality - Determination of 33 elements by inductively coupled plasma atomic emission spectrometry (Качество воды. Определение 33 элементов атомно-эмиссионной  ией с индуктивно связанной плазмой). U-образный стеклянный капилляр  4.  ,  , ', 'entities': [[218, 248, 'EQUIPMENT']]}, {'text': '-  результат  анализа  Х  (мг/куб.  дм), дельта (%), Р = 0,95, Аналитические весы ВЛА-200  Осторожно растворяют 100 г гидрооксида натрия в воде и разбавляют до 1 дм. ', 'entities': [[63, 89, 'EQUIPMENT']]}, {'text': 'При содержании фенола и ЭХГ более 0,2 мг/дм” проводится соответствующее Гайки накидные с прокладками из витона  Растворяют металл в 20 мл горячей концентрированной . Охлаждают раствор и количественно переносят в мерную колбу вместимостью 1000 мл. Доводят объем раствора до метки водой. ', 'entities': [[72, 110, 'EQUIPMENT']]}, {'text': '9.2.1. Природные и питьевые воды. Линейка измерительная  3.4.5. Перекись водорода 37%, мед., по ГОСТ 

1994

In [46]:
nlp.to_disk("nlp_model3")

In [76]:
b = nlp.to_bytes()
with open("my_file.txt", "wb") as binary_file:
    config = nlp.config
    ss = config.to_str()
    # Write bytes to file
    binary_file.write(b)

    lang_cls = spacy.util.get_lang_class('ru')
    # nlp2 = lang_cls.from_config(ss)



In [47]:
# from spacy.lang.ru import Russian
# nlp2 = spacy.load("ru_core_news_sm").from_disk("nlp_model")
nlp2 = spacy.load("nlp_model2")

In [78]:
tmp_nlp = spacy.load("ru_core_news_sm")
nlp2 = lang_cls.from_config(tmp_nlp.config)
nlp2.from_bytes(open("my_file.txt", "rb").read())

doc = nlp2("1. ОБЩИЕ ТРЕБОВАНИЯ\n1.1. Общие требования - по ГОСТ 9733.0-83.\n2. АППАРАТУРА, МАТЕРИАЛЫ\n(раздел 2 в ред. Изменения N 1, введенного Постановлением\nГосстандарта СССР от 26.09.1986 N 2912)\n2.1. Прибор для испытания устойчивости окраски к трению, обеспечивающий перемещение смежной хлопчатобумажной ткани или элементарной пробы на расстояние 100 мм, с нагрузкой 9 Н и диаметром трущего стержня 16 мм.\n2.2. Смежная хлопчатобумажная ткань по ГОСТ 9733.0-83.\n2.3. Серые шкалы для оценки закрашивания и изменения окраски по ГОСТ 9733.0-83.\n3. ОТБОР И ПОДГОТОВКА ПРОБ\n(раздел 3 в ред. Изменения N 1, введенного Постановлением\nГосстандарта СССР от 26.09.1986 N 2912")

print([(ent.text, ent.label_) for ent in doc.ents])

[('Прибор для испытания устойчивости окраски к трению, обеспечивающий перемещение смежной хлопчатобумажной ткани или элементарной пробы на расстояние 100 мм, с нагрузкой 9 Н и диаметром трущего стержня 16 мм.\n', 'EQUIPMENT'), ('Смежная хлопчатобумажная ткань по ГОСТ 9733.0-83.\n', 'EQUIPMENT')]


In [113]:
import os
def cearch_equipment(name_gost):
    path = name_gost
    if not os.path.isfile(path):
        print(f"File {path} don't exist!")
        return []
    res = []
    nlp_best = spacy.load("nlp_model1")
    for s in open(path, "r").readlines():
        # print(1)
        if len(s) > 4:
            doc = nlp_best(s)
            # morph = pymorphy2.MorphAnalyzer()
            for ent in doc.ents:
                res.append(ent.text)


    return res


In [122]:
import os
list_of_gosts = os.listdir("./gosts")
for i in range(len(list_of_gosts)):
    list_of_gosts[i] = "./gosts/" + list_of_gosts[i]

json_gosts = {}
cn = 1
for i in list_of_gosts:
    json_gosts[i] = cearch_equipment(i)
    # print(i, *json_gosts[i][:15], end='\n\n')
    print(cn, i)
    cn+=1
    # r = input()
    # if r == "-":
    #     break
    f = open("json_gosts.json", "w")
    jj = json.dumps(json_gosts)
    f.write(jj)
    f.close()

1 ./gosts/ГОСТ 4152-89.txt
2 ./gosts/МУК 4.1.1272-03.txt
3 ./gosts/ГОСТ 31870-2012.txt
4 ./gosts/ГОСТ 31949-2012.txt
5 ./gosts/ПНД Ф 14.1_2_4.139-98.txt
6 ./gosts/МУК 4.1.647-96.txt
File ./gosts/.ipynb_checkpoints don't exist!
7 ./gosts/.ipynb_checkpoints
8 ./gosts/МУК 4.1.1265-03.txt
9 ./gosts/ГОСТ 32075-2013.txt
10 ./gosts/МУК 4.1.1259-03.txt
11 ./gosts/ГОСТ 9733.27-83.txt
12 ./gosts/СанПиН 2.4.7.1166-02.txt
13 ./gosts/ГОСТ 9292-82.txt
14 ./gosts/СТБ ISO 15586-2011.txt
15 ./gosts/ГОСТ 18165-89.txt
16 ./gosts/МУК 4.1.1478-03.txt
17 ./gosts/МУ 1.1.037-95.txt
18 ./gosts/МУК 4.1.1053-01.txt
19 ./gosts/ГОСТ Р 53485-2009.txt
20 ./gosts/МУК 4.1.1045-01.txt
21 ./gosts/МУК 4.1.617-96.txt
22 ./gosts/СТБ ISO 11885-2011.txt
23 ./gosts/СТБ ГОСТ Р 51309-2001.txt
24 ./gosts/МУК 4.1.1263-03.txt
25 ./gosts/ГОСТ 4386-89.txt
26 ./gosts/ГОСТ 9134-78.txt
27 ./gosts/МУК 4.1.742-99.txt
28 ./gosts/ГОСТ 126-79.txt
29 ./gosts/ГОСТ 413-91.txt
30 ./gosts/МУК 4.1.1258-03.txt
31 ./gosts/Инструкция 2.3.3.10-15-64-

In [177]:
# !pip install fuzzywuzzy
import random
import json
from fuzzywuzzy import fuzz

#####################################################
df = pd.read_csv('standarts_list.csv')
# print(df.shape)
col = df.columns
a = [""] * df.shape[0]
for i in range(df.shape[0]):
    a[i] = df[col[0]][i].split('"')[0].strip() if df[col[0]][i].split('"')[0] != '' else df[col[0]][i]
df["add"] = a
col = df.columns

df = df.drop(columns=col[0])
df
col = df.columns
ss = list(set(df[col[0]]))
a = [""] * len(ss)
for i in range(len(ss)):
    for j in range(df.shape[0]):
        if ss[i] == df[col[0]][j]:
            if len(a[i]) == 0:
                a[i] = df[col[1]][j]
            else:
                a[i] = a[i] + ";" + df[col[1]][j]
# print(*a, sep='\n')
d = {
    "Группа продукции": ss,
    "ГОСТы": a
}
new_df_standarts_list = pd.DataFrame(data=d)

########################################################

df = pd.read_csv('standarts.csv')
print(df.shape)
col = df.columns
a = [""] * df.shape[0]
for i in range(df.shape[0]):
    a[i] = df[col[-1]][i].split('"')[0].strip() if df[col[-1]][i].split('"')[0] != '' else df[col[-1]][i]
df["add"] = a
print(list(col[0:1]) + list(col[2:6]))
df = df.drop(columns=list(col[0:1]) + list(col[2:6]))
# df
col = df.columns
ss = list(set(df[col[0]]))
a = [""] * len(ss)
for i in range(len(ss)):
    for j in range(df.shape[0]):
        if ss[i] == df[col[0]][j]:
            if len(a[i]) == 0:
                a[i] = df[col[1]][j]
            else:
                a[i] = a[i] + ";" + df[col[1]][j]
# print(*a, sep='\n')
d = {
    "Группа продукции": ss,
    "ГОСТы": a
}
new_df_standarts = pd.DataFrame(data=d)
new_col = new_df_standarts.columns

########################################################

col = new_df_standarts.columns
d = {
    "Группа продукции": list(new_df_standarts_list[col[0]]) + list(new_df_standarts[col[0]]),
    "ГОСТы": list(new_df_standarts_list[col[1]]) + list(new_df_standarts[col[1]])
}
DF = pd.DataFrame(data=d)
#################################################

df = pd.read_csv('Test - ЯЖМЫ.csv')
print(df.shape)
col = df.columns
df

s = set(df[col[2]])
f = open("key.json", "r")
key = json.load(f)
f.close()
f = open("key-gosts.json", "r")
gosts_ob = json.load(f)
f.close()
g = []
for i in gosts_ob.items():
    g.append((i[0][8:].replace(".txt", ''), i[1]))
    # print(g[-1])
gosts_ob = dict(g)
#######################################################3


for i in range(df.shape[0]):
    # df[col[-2]][i] = "Неверно"
    s = ''
    gr = ''
    for j in key:
        if j["key"] == df[col[2]][i]:
            gr = j["value"]
    # print(gr)
    gosts = []
    for j in range(DF.shape[0]):
        # print(DF[DF.columns[0]][j])
        if fuzz.WRatio(DF[DF.columns[0]][j].strip(), gr.strip()) >= 90:
              # print(12)
              gosts=DF[DF.columns[1]][j].split(";")

    for j in gosts:
        for k in gosts_ob:
            if k in j:
                for l in gosts_ob[k]:
                    s = s + l.strip() + ";"
    s = s[:-1]
    # print(s)
    # break
    if s == "":
        df[col[-2]][i] = "Верно"
    else:
        df[col[-2]][i] = "Неверно"
        df[col[-1]][i] = s


df.head()
df.to_csv("Test - ЯЖМЫ.csv", index=False)

(1492, 6)
['N п/п из Перечня', 'Структурный элемент или объект', 'Пункт/статья в ТР ТС', 'Пункт/раздел стандарта', 'Обозначение и наименование стандарта']
(58, 8)


,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Укажите верно ли указано техническое оборудование?,Ваш вариант технического оборудования
0,1,"ГОСТ 10581-91, ГОСТ 32119-2013",Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя для детей до одного года с ве...,6209909000,U-образный стеклянный капилляр; Аналитические ...,Неверно,Допустимые;Перповский А.Л.;Прибор для испытани...
1,2,ГОСТ 19245-93,"Коляски детские, комплектующие узлы и детали к...",Коляски детские комбинированные с открытыми и ...,8715009000; 8715001000,Прибор для испытания устойчивости окраски к тр...,Неверно,Загорским филиалом ВНИИЭМИ;Ткани с резиновым и...
2,3,"ГОСТ 25294-2003, ГОСТ 10581-91, ГОСТ 30386-95,...",Изделия на подкладке или без подкладки (пальто...,Изделия швейные (3-ий слой) для эпизодической ...,6204,U-образный стеклянный капилляр; Аналитические ...,Верно,NaN
3,4,ГОСТ 25295-2003,Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя верхняя швейная пальтового ас...,6209300000,U-образный стеклянный капилляр; Аналитические ...,Верно,NaN
4,5,"ГОСТ 25295-2003, ГОСТ 25295-2003, ГОСТ 25295-2...",Изделия на подкладке или без подкладки (пальто...,Изделия 2-го слоя верхние швейные костюмного а...,6203423500; 6204623900; 6204629000,U-образный стеклянный капилляр; Аналитические ...,Неверно,Прибор для испытания устойчивости окраски к тр...
5,6,"ГОСТ 29235-91 (ИСО 6742-2-85), ГОСТ 28765-90 (...",Велосипеды с высотой седла от 435 мм до 635 мм...,Велосипеды двухколесные из металла с элементам...,8712003000,маркировочное устройство; тарированный спидоме...,Верно,NaN
6,7,ГОСТ 30407-96,"Посуда и столовые приборы (чашки, блюдца, поил...",Посуда из стеклокерамики,7013100000,pH-метр любой модели; Весы лабораторные по ГОС...,Неверно,колбы;Межгосударственная система стандартизаци...
7,8,ГОСТ 31409-2009,"Пальто, куртки и аналогичные изделия, заявленн...",Изделия верхние трикотажные третьего слоя для ...,6102209000,U-образный стеклянный капилляр; Аналитические ...,Неверно,Прибор для испытания устойчивости окраски к тр...
8,9,"ГОСТ 31410-2009 , ГОСТ 31409-2009","Свитеры, джемперы, жакеты, юбки, брюки, костюм...",Изделия верхние трикотажные второго слоя для д...,6110; 6106; 6104; 6103,Хроматографическая колонка; капилляры; Весы ан...,Неверно,Допустимые;Перповский А.Л.;Прибор для испытани...
9,10,ГОСТ 32118-2013,Головные уборы (летние) 1-го слоя и аналогичны...,Головные уборы (летние) 1-го слоя,6505009000,U-образный стеклянный капилляр; Аналитические ...,Верно,NaN


In [ ]:
list_gosts = os.listdir("./gosts")
for i in range(len(list_gosts)):
    list_gosts[i] = "./gosts/" + list_gosts[i]

for i in list_gosts:
    s = open(i, "r").read()
    a = []
